In [3]:
import pyspark as ps
sc = ps.SparkContext()

In [4]:
import os

# need to get local path since we are reading local files
cwd = os.getcwd()

In [5]:
file_rdd = sc.textFile('file://' + cwd + '/data/toy_data.txt')
file_rdd.take(2)

[u'{"Jane": "2"}', u'{"Jane": "1"}']

In [30]:
# default OSX block size is 4kb
file_rdd.getNumPartitions()

11

In [8]:
!ls -larth data/toy_data.txt

-rwxr-xr-x@ 1 jonathandinu  staff   136B Aug  5 14:12 data/toy_data.txt


In [17]:
cat data/toy_data.txt

{"Jane": "2"}
{"Jane": "1"}
{"Pete": "20"}
{"Tyler": "3"}
{"Duncan": "4"}
{"Yuki": "5"}
{"Duncan": "6"}
{"Duncan": "4"}
{"Duncan": "5"}


In [33]:
file_rdd.mapPartitionsWithIndex(lambda i, iterator: (i, list(iterator))).collect()

[0,
 [u'{"Jane": "2"}',
  u'{"Jane": "1"}',
  u'{"Pete": "20"}',
  u'{"Tyler": "3"}',
  u'{"Duncan": "4"}'],
 1,
 [u'{"Yuki": "5"}',
  u'{"Duncan": "6"}',
  u'{"Duncan": "4"}',
  u'{"Duncan": "5"}']]

In [38]:
file_rdd = sc.textFile('file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt', 10)

In [39]:
# some partitions are empty, can't split a single line
file_rdd.mapPartitionsWithIndex(lambda i, iterator: (i, list(iterator))).collect()

[0,
 [u'{"Jane": "2"}'],
 1,
 [u'{"Jane": "1"}'],
 2,
 [u'{"Pete": "20"}'],
 3,
 [u'{"Tyler": "3"}'],
 4,
 [u'{"Duncan": "4"}'],
 5,
 [u'{"Yuki": "5"}'],
 6,
 [u'{"Duncan": "6"}'],
 7,
 [u'{"Duncan": "4"}'],
 8,
 [],
 9,
 [u'{"Duncan": "5"}'],
 10,
 []]

In [40]:
file_rdd = sc.textFile('file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt', 1)

In [41]:
file_rdd.mapPartitionsWithIndex(lambda i, iterator: (i, list(iterator))).collect()

[0,
 [u'{"Jane": "2"}',
  u'{"Jane": "1"}',
  u'{"Pete": "20"}',
  u'{"Tyler": "3"}',
  u'{"Duncan": "4"}',
  u'{"Yuki": "5"}',
  u'{"Duncan": "6"}',
  u'{"Duncan": "4"}',
  u'{"Duncan": "5"}']]

In [61]:
tup_rdd = file_rdd.map(lambda line: json.loads(line)) \
                  .map(lambda d: (d.keys()[0], int(d.values()[0])))

out = tup_rdd.groupByKey().mapValues(lambda tup: max(tup.data)) 

In [50]:
print tup_rdd.toDebugString()

(2) PythonRDD[78] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
 |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


In [51]:
print out.toDebugString()

(2) PythonRDD[77] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[76] at mapPartitions at PythonRDD.scala:346 []
 |  ShuffledRDD[75] at partitionBy at NativeMethodAccessorImpl.java:-2 []
 +-(2) PairwiseRDD[74] at groupByKey at <ipython-input-48-5dd19bb9f251>:3 []
    |  PythonRDD[73] at groupByKey at <ipython-input-48-5dd19bb9f251>:3 []
    |  MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
    |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


In [53]:
print file_rdd.toDebugString()

(2) MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
 |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


In [59]:
# doesn't matter if 1 or 100 map transformations
print file_rdd.map(lambda line: json.loads(line)).toDebugString()

(2) PythonRDD[85] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
 |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


In [63]:
print tup_rdd.toDebugString()

(2) PythonRDD[95] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
 |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


In [60]:
print tup_rdd.groupByKey().toDebugString()

(2) PythonRDD[90] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[89] at mapPartitions at PythonRDD.scala:346 []
 |  ShuffledRDD[88] at partitionBy at NativeMethodAccessorImpl.java:-2 []
 +-(2) PairwiseRDD[87] at groupByKey at <ipython-input-60-62a481cc790d>:1 []
    |  PythonRDD[86] at groupByKey at <ipython-input-60-62a481cc790d>:1 []
    |  MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
    |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


In [66]:
print out.toDebugString()

(2) PythonRDD[96] at RDD at PythonRDD.scala:43 []
 |  MapPartitionsRDD[94] at mapPartitions at PythonRDD.scala:346 []
 |  ShuffledRDD[93] at partitionBy at NativeMethodAccessorImpl.java:-2 []
 +-(2) PairwiseRDD[92] at groupByKey at <ipython-input-61-5dd19bb9f251>:3 []
    |  PythonRDD[91] at groupByKey at <ipython-input-61-5dd19bb9f251>:3 []
    |  MapPartitionsRDD[71] at textFile at NativeMethodAccessorImpl.java:-2 []
    |  file:///Users/jonathandinu/Repositories/pearson/data-applications-sprk/clean_notebooks/data/toy_data.txt HadoopRDD[70] at textFile at NativeMethodAccessorImpl.java:-2 []


## Scala

```scala
file_rdd.mapPartitionsWithIndex((ind, part) => part.map(x => (ind, x))).collect()

file_rdd.map(line => line.split(" "))
        .map(split => (split(0), split(1).toInt))
        .groupByKey()
        .mapValues(iter => iter.reduce(_ + _)).collect()

file_rdd.map(line => line.split(" ")).map(split => (split(0), split(1).toInt))
                                     .groupByKey()
                                     .mapPartitionsWithIndex((ind, part) => part.map(x => (ind, x)))
                                     .collect()
                                     
// res68: Array[(Int, (String, Iterable[Int]))] = Array((0,(anna,CompactBuffer(1))), (0,(jesse,CompactBuffer(3))), (1,(jon,CompactBuffer(2, 1))), (1,(mary,CompactBuffer(3, 5))))
```